In [3]:
import torch
import torch.nn as nn
from math import ceil

In [4]:
## cnn block

## sequeeze excietation

## inverted residual block

class DeepConv(nn.Module):

    def __init__(self, i, o, k, s, p, g):
        super(DeepConv, self).__init__()
        self.conv = nn.Conv2d(i, o, k, s, p, g, bias=False)
        self.norm = nn.BatchNorm2d(o)
        self.act = nn.SELU()
    
    def forward(self, x):
        return self.act(self.norm(self.conv(x)))


class SqueezeEx(nn.Module):

    def __init__(self, i, reduced_dim):
        super(SqueezeEx, self).__init__()
        self.squeeze = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(i, reduced_dim, 1), 
            nn.SiLU(), 
            nn.Conv2d(reduced_dim, i, 1), 
            nn.Sigmoid()
        )

    def forward(self, x):
        return x * self.squeeze(x)


class InvertedResidualBlock(nn.Module):

    def __init__(self, i, o, k, s, p, expand_ratio, reduction_dim=4, droppath=0.5):
        super(InvertedResidualBlock, self).__init__()
        self.droppath = droppath
        self.use_residual = i == o and s == 1
        self.hidden_dim = i * expand_ratio
        self.expand = i != hiddem_dim
        reduced_dim = int(i/reduction_dim)

        if self.expand:
            self.expand_conv = DeepConv(i, self.hidden_dim, 3, 1, 1)

        self.conv = nn.Sequential(
            DeepConv(self.hidden_dim, self.hidden_dim, k, s, p, self.hidden_dim),
            SqueezeEx(self.hidden_dim, reduced_dim), 
            nn.Conv2d(self.hidden_dim, o, 1, bias=False), 
            nn.BatchNorm2d(o)
        )

    def shochastic_depth(self, x):

        if not self.training:
            return x

        binary_tensor = torch.rand(x.shape[0], 1, 1, 1, device=x.device) < self.droppath
        return torch.div(self.droppath , x) * binary_tensor

    def forward(self, inputs):
        x = self.expand_conv(x) if self.expand else inputs

        if self.use_residual:
            return self.shochastic_depth(self.conv(x)) + inputs
        else:
            return self.conv(x)




In [ ]:
## deep conv

## inverted conv